###LLMを使った、 「吾輩は猫である」に基づいたRetrieval Augmented Generation(RAG)の実装


1. [準備](#scrollTo=esq3PYWLIaK5&line=1&uniqifier=1)
2. [Generation without Retrieval](#scrollTo=6WZoYOfvIaK5)
3. [Generation with gold passage](#scrollTo=dh2HS6-LIaK6)
4. [Retrievalの実装](#scrollTo=5B8JmFzAIaK6&line=2&uniqifier=1)
5. [Retrieval Augmented Generation](#scrollTo=80b-PBPTIaK7)
6. [参考文献]()


##準備

In [ ]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 4.9 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# https://huggingface.co/cyberagent/open-calm-1b
model = AutoModelForCausalLM.from_pretrained("cyberagent/open-calm-1b", device_map="auto", torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained("cyberagent/open-calm-1b")

if torch.cuda.is_available():
    model = model.to("cuda")

config.json:   0%|          | 0.00/610 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.94G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/323 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.23M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

In [ ]:
!nvidia-smi

Thu Nov 23 01:56:54 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P0    28W /  70W |   3503MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Generation without Retrieval

In [ ]:
prompt = """以下の質問に回答してください:
質問:　吾輩が指すものは何ですか。
回答:"""

token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

with torch.no_grad():
    output_ids = model.generate(
        token_ids.to(model.device),
        max_new_tokens=100,
        min_new_tokens=10,
        do_sample=True,
        temperature=0.7,
        top_p=0.95,
        repetition_penalty=1.02,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(output)

以下の質問に回答してください:
質問: 吾輩が指すものは何ですか。
回答: それは私どものサービスであり、あなた方のために働くものです<|endoftext|>


ただ質問をしただけでは事実に基づいた正しい回答が得られない

### Generation with gold passage

In [ ]:
query = "吾輩が指すものは何ですか。"
retrieved_text = """吾輩は猫である。
"""

prompt = f"""{retrieved_text}
上記の文章のみをもとにして質問に回答してください。一歩ずつ考えましょう。
質問: {query}
回答:"""
print(prompt)

吾輩は猫である。

上記の文章のみをもとにして質問に回答してください。一歩ずつ考えましょう。
質問: 吾輩が指すものは何ですか。
回答:


In [ ]:
token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

with torch.no_grad():
    output_ids = model.generate(
        token_ids.to(model.device),
        max_new_tokens=100,
        min_new_tokens=10,
        do_sample=True,
        temperature=0.7,
        top_p=0.95,
        repetition_penalty=1.02,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(output)

吾輩は猫である。

上記の文章のみをもとにして質問に回答してください。一歩ずつ考えましょう。
質問: 吾輩が指すものは何ですか。
回答: 「ねこ」です。「ねこの顔の形をした箱」「穴の空いたガラス瓶の中に入っているもの」=「小動物」。それを一匹でも持っていると、その人は(多分)とても幸せになれるそうです。(他の回答者さんのコメントで知りました。)<|endoftext|>


回答の根拠となる文章が与えられると、なんとか正解できる

### Retrieval の実装
「吾輩は猫である」の文章内を検索して質問に答える流れを実装する

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Portfolio/LLM/neco.txt', header=None, names=['content'])
df.head()

,content
0,吾輩（わがはい）は猫である。名前はまだ無い。
1,どこで生れたかとんと見当（けんとう）がつかぬ。何でも薄暗いじめじめした所でニャーニャー泣いて...
2,この書生の掌の裏（うち）でしばらくはよい心持に坐っておったが、しばらくすると非常な速力で運転...
3,ふと気が付いて見ると書生はいない。たくさんおった兄弟が一疋（ぴき）も見えぬ。肝心（かんじん）...
4,ようやくの思いで笹原を這い出すと向うに大きな池がある。吾輩は池の前に坐ってどうしたらよかろう...


文章をベクトル化し、質問文と文章の類似度を計算する

In [ ]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.0 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=e229ff6cc9ade6baed145f28d820b65493e56025076fbebeacb664df1fa22373
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [ ]:
from sentence_transformers import SentenceTransformer

# https://huggingface.co/intfloat/multilingual-e5-large
model = SentenceTransformer('intfloat/multilingual-e5-large')

.gitattributes:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

In [ ]:
input_texts = ['passage: ' + content for content in df.content.tolist()]
input_texts

['passage: 吾輩（わがはい）は猫である。名前はまだ無い。',
 'passage: どこで生れたかとんと見当（けんとう）がつかぬ。何でも薄暗いじめじめした所でニャーニャー泣いていた事だけは記憶している。吾輩はここで始めて人間というものを見た。しかもあとで聞くとそれは書生という人間中で一番獰悪（どうあく）な種族であったそうだ。この書生というのは時々我々を捕（つかま）えて煮（に）て食うという話である。しかしその当時は何という考もなかったから別段恐しいとも思わなかった。ただ彼の掌（てのひら）に載せられてスーと持ち上げられた時何だかフワフワした感じがあったばかりである。掌の上で少し落ちついて書生の顔を見たのがいわゆる人間というものの見始（みはじめ）であろう。この時妙なものだと思った感じが今でも残っている。第一毛をもって装飾されべきはずの顔がつるつるしてまるで薬缶（やかん）だ。その後（ご）猫にもだいぶ逢（あ）ったがこんな片輪（かたわ）には一度も出会（でく）わした事がない。のみならず顔の真中があまりに突起している。そうしてその穴の中から時々ぷうぷうと煙（けむり）を吹く。どうも咽（む）せぽくて実に弱った。これが人間の飲む煙草（たばこ）というものである事はようやくこの頃知った。',
 'passage: この書生の掌の裏（うち）でしばらくはよい心持に坐っておったが、しばらくすると非常な速力で運転し始めた。書生が動くのか自分だけが動くのか分らないが無暗（むやみ）に眼が廻る。胸が悪くなる。到底（とうてい）助からないと思っていると、どさりと音がして眼から火が出た。それまでは記憶しているがあとは何の事やらいくら考え出そうとしても分らない。',
 'passage: ふと気が付いて見ると書生はいない。たくさんおった兄弟が一疋（ぴき）も見えぬ。肝心（かんじん）の母親さえ姿を隠してしまった。その上今（いま）までの所とは違って無暗（むやみ）に明るい。眼を明いていられぬくらいだ。はてな何でも容子（ようす）がおかしいと、のそのそ這（は）い出して見ると非常に痛い。吾輩は藁（わら）の上から急に笹原の中へ棄てられたのである。',
 'passage: ようやくの思いで笹原を這い出すと向うに大きな池がある。吾輩は池の前に坐ってどうしたらよかろうと考えて見た。別にこれという分別（ふんべつ）も出ない。し

### Retrieval Augmented Generation

In [ ]:
query = "吾輩が指すものは何ですか。"
query_embeddings = model.encode(['query: ' + query], normalize_embeddings=True)
passage_embeddings = model.encode(input_texts, normalize_embeddings=True)
scores = (query_embeddings @ passage_embeddings.T) * 100

# 検索結果上位3件
print('score: ', scores[0][scores[0].argsort()[::-1][0]])
print(input_texts[scores[0].argsort()[::-1][0]])
print('score: ', scores[0][scores[0].argsort()[::-1][1]])
print(input_texts[scores[0].argsort()[::-1][1]])
print('score: ', scores[0][scores[0].argsort()[::-1][2]])
print(input_texts[scores[0].argsort()[::-1][2]])

score:  86.88329
passage: 吾輩（わがはい）は猫である。名前はまだ無い。
score:  85.54552
passage: 吾輩はまた少々休養を要する。主人と多々良君が上野公園でどんな真似をして、芋坂で団子を幾皿食ったかその辺の逸事は探偵の必要もなし、また尾行（びこう）する勇気もないからずっと略してその間（あいだ）休養せんければならん。休養は万物の旻天（びんてん）から要求してしかるべき権利である。この世に生息すべき義務を有して蠢動（しゅんどう）する者は、生息の義務を果すために休養を得ねばならぬ。もし神ありて汝（なんじ）は働くために生れたり寝るために生れたるに非ずと云わば吾輩はこれに答えて云わん、吾輩は仰せのごとく働くために生れたり故に働くために休養を乞うと。主人のごとく器械に不平を吹き込んだまでの木強漢（ぼくきょうかん）ですら、時々は日曜以外に自弁休養をやるではないか。多感多恨にして日夜心神を労する吾輩ごとき者は仮令（たとい）猫といえども主人以上に休養を要するは勿論の事である。ただ先刻（さっき）多々良君が吾輩を目して休養以外に何等の能もない贅物（ぜいぶつ）のごとくに罵（ののし）ったのは少々気掛りである。とかく物象（ぶっしょう）にのみ使役せらるる俗人は、五感の刺激以外に何等の活動もないので、他を評価するのでも形骸以外に渉（わた）らんのは厄介である。何でも尻でも端折（はしょ）って、汗でも出さないと働らいていないように考えている。達磨（だるま）と云う坊さんは足の腐るまで座禅をして澄ましていたと云うが、仮令（たとい）壁の隙（すき）から蔦（つた）が這い込んで大師の眼口を塞（ふさ）ぐまで動かないにしろ、寝ているんでも死んでいるんでもない。頭の中は常に活動して、廓然無聖（かくねんむしょう）などと乙な理窟を考え込んでいる。儒家にも静坐の工夫と云うのがあるそうだ。これだって一室の中（うち）に閉居して安閑と躄（いざり）の修行をするのではない。脳中の活力は人一倍熾（さかん）に燃えている。ただ外見上は至極沈静端粛の態（てい）であるから、天下の凡眼はこれらの知識巨匠をもって昏睡仮死（こんすいかし）の庸人（ようじん）と見做（みな）して無用の長物とか穀潰（ごくつぶ）しとか入らざる誹謗（ひぼう）の声を立てるのである。これらの凡眼は皆形を見て心を見ざる不具なる視覚を有して生

In [ ]:
query = "吾輩が指すものは何ですか。"
query_embeddings = model.encode(['query: ' + query], normalize_embeddings=True)
scores = (query_embeddings @ passage_embeddings.T) * 100

top_k = 3
top_k_idx = scores[0].argsort()[::-1][:top_k]

retrieved_text = f"""{df.content.tolist()[top_k_idx[0]][:20]}
{df.content.tolist()[top_k_idx[1]][:20]}
{df.content.tolist()[top_k_idx[2]][:20]}
"""

prompt = f"""{retrieved_text}
上記の文章のみをもとにして質問に回答してください。一歩ずつ考えましょう。
質問: {query}
回答:"""
print(prompt)

吾輩（わがはい）は猫である。名前はまだ無
吾輩はまた少々休養を要する。主人と多々良
吾輩は猫である。猫の癖にどうして主人の心

上記の文章のみをもとにして質問に回答してください。一歩ずつ考えましょう。
質問: 吾輩が指すものは何ですか。
回答:


In [ ]:
model = AutoModelForCausalLM.from_pretrained("cyberagent/open-calm-1b", device_map="auto", torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained("cyberagent/open-calm-1b")

if torch.cuda.is_available():
    model = model.to("cuda")

In [ ]:
token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

with torch.no_grad():
    output_ids = model.generate(
        token_ids.to(model.device),
        max_new_tokens=100,
        min_new_tokens=10,
        do_sample=True,
        temperature=0.7,
        top_p=0.95,
        repetition_penalty=1.02,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(output)

吾輩(わがはい)は猫である。名前はまだ無
吾輩はまた少々休養を要する。主人と多々良
吾輩は猫である。猫の癖にどうして主人の心

上記の文章のみをもとにして質問に回答してください。一歩ずつ考えましょう。
質問: 吾輩が指すものは何ですか。
回答: 「御仏」です。「おぼとけ」。<|endoftext|>


##参考文献
In-Context Retrieval-Augmented Language Models: https://arxiv.org/abs/2302.00083